# Evaluation Model 3
## Model name : unsloth/mistral-7b-v0.3
## Link Colab : [Click](https://colab.research.google.com/drive/1qPa0zKxjyVTefuoOnfZbSMcDdIs3mcJi?usp=sharing)
## Link Adapter weight : [Click](https://drive.google.com/file/d/1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0/view?usp=drive_link)

# Download Dependencies

In [ ]:
%%capture
! pip install unsloth
! pip install rouge_score
! pip install evaluate

In [ ]:
%%capture
! pip install -U peft
! pip install bert_score

# Import Dependencies

In [ ]:
from tqdm import tqdm
import kagglehub
import os

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import evaluate

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from datasets import load_dataset
import evaluate
import torch
import bert_score
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import os
import re

In [ ]:
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset

In [ ]:
from peft import PeftModel

# Load dataset

In [ ]:
# 1. Load dataset
dataset = load_dataset("farwew/DoctorsAnswerTextDataset-in-Indonesian")

README.md:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

(…)-00000-of-00001-e657306e4ba4079a.parquet:   0%|          | 0.00/17.9M [00:00<?, ?B/s]

(…)-00000-of-00001-6bdfe08bb1c69cf1.parquet:   0%|          | 0.00/948k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19845 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1048 [00:00<?, ? examples/s]

In [ ]:
# 3. Ambil data test
dataset_eval = dataset["test"]

# 4. Konversi ke pandas DataFrame
df = dataset_eval.to_pandas()
df.head()

,instruction,input,output,topic_set,topics
0,Selalu tersedak setiap makan dan minum,Selalu tersedak setiap makan dan minum. Dok sa...,batuk adalah respon tubuh untuk mengeluarkan b...,gastritis,penyakit-parkinson multiple-sclerosis gastritis
1,Apa obat untuk demam?,Apa obat untuk demam?. malam dok saya mau tany...,"kepala pusing, perut mual, punggung sakit dan ...",mual,flu pilek sakit-kepala nyeri-punggung mual
2,Cara mengatasi perut kembung pada anak,Cara mengatasi perut kembung pada anak. punya ...,perut kembung menandakan adanya penumpukan gas...,perut-kembung,anak gastritis perut-kembung
3,Perut Bagian Bawah Kembung,"Perut Bagian Bawah Kembung. saya mau bertanya,...","seperti: sayur: kol, brokoli, kembang kol, asp...",gangguan-pencernaan,gangguan-pencernaan
4,Nyeri dada menjalar ke punggung,Nyeri dada menjalar ke punggung. Malam dok tad...,saya coba bantu jawab. keluhan nyeri dada menj...,sakit-maag,nyeri-dada sakit-maag


In [ ]:
# 5. Stratified split berdasarkan kolom 'topic_set'
train_df, stratified_sample_df = train_test_split(
    df,
    test_size=100,  # ubah sesuai proporsi yang diinginkan
    stratify=df["topic_set"],
    random_state=42
)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
stratified_dataset = Dataset.from_pandas(stratified_sample_df)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

# Load Adapter

In [ ]:
%%capture
!pip install gdown

In [ ]:
! gdown 'https://drive.google.com/uc?id=1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0'

Downloading...
From (original): https://drive.google.com/uc?id=1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0
From (redirected): https://drive.google.com/uc?id=1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0&confirm=t&uuid=21908ca7-1507-43f3-bbff-6d0d748024e0
To: /content/1 epoch -  Mistral_v0.3_ instruct (7B) - Alpaca (run 3).zip
100% 78.4M/78.4M [00:01<00:00, 60.9MB/s]


In [ ]:
! unzip "/content/1 epoch -  Mistral_v0.3_ instruct (7B) - Alpaca (run 3).zip"

Archive:  /content/1 epoch -  Mistral_v0.3_ instruct (7B) - Alpaca (run 3).zip
   creating: lora_model/
  inflating: lora_model/tokenizer.model  
  inflating: lora_model/tokenizer.json  
  inflating: lora_model/README.md    
  inflating: lora_model/adapter_config.json  
  inflating: lora_model/tokenizer_config.json  
  inflating: lora_model/adapter_model.safetensors  
  inflating: lora_model/special_tokens_map.json  


#  Notebook Evaluasi Base Model (tanpa Adapter LoRA)

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.5.7: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Mis

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_func(examples):
    return {
        "text": [
            alpaca_prompt.format(ins, inp, out) + EOS_TOKEN
            for ins, inp, out in zip(examples["instruction"], examples["input"], examples["output"])
        ]
    }

## Inference dengan base model

In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "mencegah mual",
            "Anak saya mengalami mual, bagaimana mencegah mual?",
            ""
        )
    ],
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
mencegah mual

### Input:
Anak saya mengalami mual, bagaimana mencegah mual?

### Response:
Mual adalah gejala yang mungkin diakibatkan oleh beberapa faktor, seperti makanan yang tidak sesuai, kekurangan cahaya atau tidur buruk. Untuk mencegah mual, Anda dapat mengikuti beberapa langkah sebagai berikut:

1. Jangan makan makanan yang tidak sesuai, seperti makanan yang mengandung banyak gula atau minyak.
2. Jangan minum susu atau minum teh setelah makan.
3. Jadilah tidak buruk, sekali gak makan lembap, dan tidak minum cairan yang panas.
4. Jangan tinggalkan diri Anda terlalu lama di dekat layar komputer atau televisi.
5. Jangan tinggalkan diri Anda terlalu lama tanpa makan atau minum.
6. Jangan ragu-ragu, coba tidur sejak waktu yang tepat.
7. Jangan lupa untuk melakukan kegiatan fisik sehari-hari.

Sekian, semoga l

## Evaluation

In [ ]:
# === Evaluasi === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_instructions = []
all_inputs = []

In [ ]:
def normalize_text(text):
    text = text.lower().strip()
    text = re.sub(r"\p{P}+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Base Model"):
    prompt = alpaca_prompt.format(example["instruction"], example["input"], "")
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    if num < 10:
        print(prompt)
        print()
        print('response:',response)

    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])
    num+=1

Evaluating Base Model:   1%|          | 1/100 [00:17<29:32, 17.91s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solusi atasi sakit maag yang tak kunjung sembuh

### Input:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam, akibatnya saya mengalami muntah berwarna kekuningan disertai dengan bercak merah seperti darah segar. A

Evaluating Base Model:   2%|▏         | 2/100 [00:40<33:52, 20.74s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
solusi untuk mengatasi konstipasi

### Input:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Response:


response: Masalah konstipasi yang anda alami dapat disebabkan oleh beberapa faktor, seperti kekurangan minyak lemak atau air dalam makanan anda, tidak makan cukup atau tidak bergerak cukup. Untuk mengatasi masalah ini, anda dapat mencoba beberapa hal seperti:

1. Minum air secukupnya setiap hari untuk memastikan anda terhindar dari dehidrasi.
2. Makanan yang mengandung fiber tinggi seperti sayuran, buah-buahan, dan jagung.
3. Bergerak secara teratur setiap hari, seperti menjalani atau berolahraga.
4. Gunakan senyawa kejutan seperti senyawa senna atau senyawa magnesium bisulfit.
5. Jangan terlalu makanan ringan atau

Evaluating Base Model:   3%|▎         | 3/100 [00:48<23:48, 14.73s/it]

...

Evaluating Base Model:   8%|▊         | 8/100 [02:47<34:39, 22.60s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
sakit perut sebelah kiri bawah

### Input:
sakit perut sebelah kiri bawah. Dok, saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari selesai saya haid tanggal 30 maret 2016. Sakitnya sampai ke pinggang dan tanda tanda nya seperti akan haid. Karena pada saya pun terasa nyeri dan cenderung lebih sensitif. Di minggu ketiga ini sakitnya terasa ngilu sampai ke paha kiri. Bahkan kadang panggul panggul pun sakit. Itu kenapa ya dok?apakah saya hamil? Terima kasih

### Response:


response: Masukan pertanyaan Anda ke dalam alat pemeliharaan kesehatan Anda sebelum meningkatkan rasa susah hati. Saya tidak dapat memberikan persyaratan hamil melalui komunikasi teks saja. Namun, Anda menyebutkan gejala-gejala yang mirip dengan gejala haid yang sedang

Evaluating Base Model:   9%|▉         | 9/100 [03:22<40:08, 26.47s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Nyeri Ulu Hati Pasca Operasi Usus buntu

### Input:
Nyeri Ulu Hati Pasca Operasi Usus buntu. sudah 4 Bulan yang lalu saya di operasi usus buntu (Operasi Laparotomi) minggu minggu ini perut saya aga kurang enak dok. tadi pagi setelah BAB perut saya kram,melilit,perih....dan nyeri Ulu ati, nyerinya timbul tenggelam...tapi setelah minum obat maag (Omeprazol) sakitnya mereda tapi masih aga kurang enak perutnya itu penyebabnya apa ya dok? apa karena Maag atau ada iritasi di Usus? atau ada hubungannya dengan operasi usus buntu 4 bulan yang lalu?

### Response:


response: Saya percaya bahwa Anda mengalami peristiwa yang sering disebut sebagai "post-laparotomy syndrome" atau "post-surgical syndrome". Ini adalah sekumpulan gejala yang sering terjadi setelah perjalanan jangka panjang seperti operasi usus, dan gejala-gejal

Evaluating Base Model: 100%|██████████| 100/100 [31:21<00:00, 18.82s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/base_model_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI BASE MODEL SELESAI ===")
print(metrics)

=== EVALUASI BASE MODEL SELESAI ===
{'bert_score': {'precision': 65.17, 'recall': 63.12, 'f1': 64.1}, 'bleu': 0.59, 'rouge': {'rouge1': np.float64(13.84), 'rouge2': np.float64(1.86), 'rougeL': np.float64(7.2)}}


In [ ]:
hasil_base = pd.read_csv(f"{output_dir}/base_model_outputs.csv")
hasil_base

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,dada bagian tengah terasa tidak nyaman,dada bagian tengah terasa tidak nyaman. saya m...,kondisi yang anda alami saat ini dengan keluha...,"Based on your description, it seems that you'r...",0.626971,0.601298,0.613866
1,Solusi atasi sakit maag yang tak kunjung sembuh,Solusi atasi sakit maag yang tak kunjung sembu...,usus halus ataupun kerongkongan yang dapat dis...,"Based on the information provided, it appears ...",0.629694,0.596360,0.612574
2,solusi untuk mengatasi konstipasi,solusi untuk mengatasi konstipasi. dok kalo ba...,konstipasi adalah keadaan tidak dapat buang ai...,Masalah konstipasi yang anda alami dapat diseb...,0.674171,0.653103,0.663469
3,"Perut sakit, mual dan bersendawa berlebihan","Perut sakit, mual dan bersendawa berlebihan. s...","penyebab sakit perut, mual dan bersendawa mung...",I'm sorry to hear that your wife is experienci...,0.626235,0.580974,0.602756
4,Perut sering sakit di sertai mual,Perut sering sakit di sertai mual. Maaf dok' k...,sebaiknya anda berkonsultasi dengan dokter di ...,"Based on the symptoms you've described, it's p...",0.575403,0.580630,0.578005
...,...,...,...,...,...,...,...
96,Muntah-muntah pada penderita infeksi lambung a...,Muntah-muntah pada penderita infeksi lambung a...,lambung adalah organ yang vital fungsinya dala...,"Selama perawatan di rumah sakit, kejangan yang...",0.715448,0.649979,0.681144
97,"Demam tinggi, mual, dan tidak nafsu makan","Demam tinggi, mual, dan tidak nafsu makan. do...","kolera, disentri dulu. radang usus = kolitis ,...","The symptoms you described - high fever, dry m...",0.621983,0.606468,0.614127
98,Perbedaan serangan jantung dan lambung,Perbedaan serangan jantung dan lambung. Dok ma...,"namun, untuk membedakan apakah serangan jantun...","Dari informasi yang Anda berikan, demam, stami...",0.670994,0.683400,0.677140
99,Penyebab BAB berdarah dan mengenai pemeriksaan...,Penyebab BAB berdarah dan mengenai pemeriksaan...,anal seks merupakan salah satu perbuatan beres...,Selamat datang ke layanan kami. Dari informasi...,0.663823,0.653177,0.658457


# Dengan Adapter

In [ ]:
import torch
import gc

# Clear PyTorch cache
torch.cuda.empty_cache()

# Garbage collection
gc.collect()

283

In [ ]:
MAX_SEQ_LENGTH = 2048
USE_4BIT = True

# === LOAD LoRA MODEL === #
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/lora_model",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = USE_4BIT,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.5.7: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load /content/lora_model as a legacy tokenizer.
Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "mencegah mual",
    "Anak saya mengalami mual, bagaimana mencegah mual?",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
mencegah mual

### Input:
Anak saya mengalami mual, bagaimana mencegah mual?

### Response:
mual merupakan suatu sensasi tidak nyaman pada perut yang disertai dengan rasa ingin muntah. mual dapat disebabkan oleh berbagai hal, diantaranya adalah: penyakit asam lambung infeksi saluran pencernaan keracunan makanan vertigo mabuk laut atau mabuk perjalanan mual pada kehamilan mabuk kendaraan mabuk makanan mabuk minuman mabuk obat-obatan efek samping obat-obatan tertentu mual pada anak dapat disebabkan oleh berbagai hal, diantaranya adalah: infeksi saluran pencernaan keracunan makanan mabuk kendaraan mabuk makanan mabuk minuman mabuk obat-obatan efek samping obat-obatan tertentu mual pada anak dapat disebabkan oleh berbagai hal, diantaranya adalah: infeksi saluran pencernaan keracunan makanan mabuk kendaraan mabuk m

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "Selalu tersedak setiap makan dan minum",
    "Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Selalu tersedak setiap makan dan minum

### Input:
Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...

### Response:
kondisi yang anda alami dengan keluhan utama berupa tersedak, dimana dalam bahasa medis disebut sebagai gerd, atau gastroesophageal reflux disease, merupakan suatu kondisi yang ditandai dengan nyeri pada ulu hati atau sensasi terbakar di dada akibat naiknya asam lambung menuju esofagus. gejala yang umumnya muncul jika seseorang menderita penyakit ini, diantaranya adalah : rasa tidak nyaman di dada, nyeri pada ulu hati, mulut yang terasa pahit, mual, muntah, kembung, sering bersendawa, dulu. selain oleh karena gerd, kondisi yang anda alami juga dapat di

In [ ]:
# === IMPORT TAMBAHAN === #
import bert_score
import pandas as pd
import json
from datetime import datetime
import os

# === LOAD METRIK EVALUASI === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_inputs = []  # Untuk menyimpan input
all_instructions = []  # Untuk menyimpan instruction

# === PARAMETER === #
batch_size = 1  # Sesuaikan dengan kapasitas GPU
MAX_DISPLAY = 10
displayed = 0

# === TIMESTAMP UNTUK NAMA FILE === #
timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"evaluation_results_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

In [ ]:
print("Total parameters:", model.num_parameters())
print("Trainable parameters:", model.get_nb_trainable_parameters())


Total parameters: 7262703616
Trainable parameters: (20971520, 7262703616)


In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Fine tune Model"):
    prompt = alpaca_prompt.format(example["instruction"], example["input"], "")
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    if num < 10:
        print(prompt)
        print()
        print('response:',response)

    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])
    num+=1

Evaluating Fine tune Model:   1%|          | 1/100 [01:09<1:54:52, 69.62s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solusi atasi sakit maag yang tak kunjung sembuh

### Input:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam, akibatnya saya mengalami muntah berwarna kekuningan disertai dengan bercak merah seperti darah segar. A

Evaluating Fine tune Model:   2%|▏         | 2/100 [01:38<1:14:25, 45.57s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
solusi untuk mengatasi konstipasi

### Input:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Response:


response: konstipasi atau sulit buang air besar merupakan kondisi dimana terjadi kesulitan atau tidak dapat buang air besar secara teratur atau tidak dapat sama sekali.konstipasi dapat disebabkan oleh beberapa hal, antara lain: pola makan yang buruk, seperti kurang minum, kurang serat, kurang aktivitas fisik kebiasaan mengabaikan keinginan buang air besar sering menunda buang air besar efek samping obat-obatan tertentu kondisi medis tertentu, seperti penyumbatan usus, penyempitan usus, penyakit parkinson, diabetes, stroke, hipotiroidisme, gangguan psikologis, dulu untuk memastikan penyebab keluhan anda, sebaiknya a

Evaluating Fine tune Model:   3%|▎         | 3/100 [02:16<1:08:11, 42.18s/it]

...

Evaluating Fine tune Model:   8%|▊         | 8/100 [04:25<39:16, 25.62s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
sakit perut sebelah kiri bawah

### Input:
sakit perut sebelah kiri bawah. Dok, saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari selesai saya haid tanggal 30 maret 2016. Sakitnya sampai ke pinggang dan tanda tanda nya seperti akan haid. Karena pada saya pun terasa nyeri dan cenderung lebih sensitif. Di minggu ketiga ini sakitnya terasa ngilu sampai ke paha kiri. Bahkan kadang panggul panggul pun sakit. Itu kenapa ya dok?apakah saya hamil? Terima kasih

### Response:


response: saya membantu menjawab pertanyaannya. sakit perut sebelah kiri bawah dapat disebabkan oleh berbagai macam hal. untuk mengetahui penyebab pasti dari keluhan yang anda rasakan, diperlukan pemeriksaan langsung oleh dokter. beberapa kondisi yang dapat menyebabkan

Evaluating Fine tune Model:   9%|▉         | 9/100 [04:54<40:21, 26.61s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Nyeri Ulu Hati Pasca Operasi Usus buntu

### Input:
Nyeri Ulu Hati Pasca Operasi Usus buntu. sudah 4 Bulan yang lalu saya di operasi usus buntu (Operasi Laparotomi) minggu minggu ini perut saya aga kurang enak dok. tadi pagi setelah BAB perut saya kram,melilit,perih....dan nyeri Ulu ati, nyerinya timbul tenggelam...tapi setelah minum obat maag (Omeprazol) sakitnya mereda tapi masih aga kurang enak perutnya itu penyebabnya apa ya dok? apa karena Maag atau ada iritasi di Usus? atau ada hubungannya dengan operasi usus buntu 4 bulan yang lalu?

### Response:


response: mungkin keluhan yang anda alami saat ini dapat disebabkan oleh gangguan pada lambung. gangguan pada lambung dapat berupa: maag penyakit asam lambung tukak lambung kondisi yang anda alami saat ini dapat saja berkaitan dengan riwayat operasi usus buntu 

Evaluating Fine tune Model: 100%|██████████| 100/100 [54:32<00:00, 32.72s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/fine_tuned_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI Fine tune MODEL SELESAI ===")
print(metrics)

=== EVALUASI Fine tune MODEL SELESAI ===
{'bert_score': {'precision': 69.34, 'recall': 69.16, 'f1': 69.21}, 'bleu': 4.64, 'rouge': {'rouge1': np.float64(31.19), 'rouge2': np.float64(6.95), 'rougeL': np.float64(16.02)}}


In [ ]:
! zip -r moedel_base_3.zip /content/eval_basemodel_20250525_025234

updating: content/eval_basemodel_20250525_025234/ (stored 0%)
  adding: content/eval_basemodel_20250525_025234/metrics.json (deflated 46%)
  adding: content/eval_basemodel_20250525_025234/base_model_outputs.csv (deflated 68%)


In [ ]:
! zip -r moedel_ft_3.zip /content/eval_basemodel_20250525_035241

  adding: content/eval_basemodel_20250525_035241/ (stored 0%)
  adding: content/eval_basemodel_20250525_035241/fine_tuned_outputs.csv (deflated 73%)
  adding: content/eval_basemodel_20250525_035241/metrics.json (deflated 47%)


In [ ]:
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 4.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.21 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 16%|█▌        | 5/32 [00:00<00:02, 10.64it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:55<00:00,  3.60s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00003.bin...
Unsloth: Saving model/pytorch_model-00002-of-00003.bin...
Unsloth: Saving model/pytorch_model-00003-of-00003.bin...
Done.


Unsloth: You are pushing to hub, but you passed your HF username = hf.
We shall truncate hf/model to model


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.04 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [01:47<00:00,  3.37s/it]


Unsloth: Saving to organization with address hf/model
Unsloth: Saving tokenizer... Done.
Unsloth: Saving to organization with address hf/model
Unsloth: Saving hf/model/pytorch_model-00001-of-00003.bin...
Unsloth: Saving hf/model/pytorch_model-00002-of-00003.bin...
Unsloth: Saving hf/model/pytorch_model-00003-of-00003.bin...
Unsloth: Uploading all files... Please wait...


RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-683297d4-294fcc9f4169643d40a54ee1;9cd55209-8300-4cff-9512-7041219b2a18)

Repository Not Found for url: https://huggingface.co/api/models/hf/model/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.

In [ ]:
if True: model.push_to_hub_merged("farwew/model_3", tokenizer, save_method = "merged_16bit", token = "hf_REMOVED_BY_SCRIPT")

Unsloth: You are pushing to hub, but you passed your HF username = farwew.
We shall truncate farwew/model_3 to model_3


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.56 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 41%|████      | 13/32 [00:52<01:15,  3.97s/it]